In [ ]:
import pandas as pd

from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

import re
import sys
import evaluate
import unicodedata
import pandas as pd

wer = evaluate.load("wer")
cer = evaluate.load("cer")

from utils import strip_diacritics, remove_punctuation

/home/hawau.toyin/miniconda3/envs/speech/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
"""
Sample extrac sentences in LLM response, using regex we extract the diacritized words from the model response.
"""

matches_qwen = [
r"diacritics and accent marks added:(.*?) Note"
r".Here's a possible rendition with diacritics:(.*?)- Òbè:",
r"Here's the text with the appropriate diacritics:(.*?)Here's",
r"Here is the text with appropriate diacritics and accent marks:(.*?)Here's",
r"Here is the text with diacritics and accent marks added:(.*?). Here Explanation of",
r"Here is the text with diacritics and accent marks:(.*?)Here's",
r"the text with diacritics could be:(.*?)This interpretation",
r"Here's a possible transcription with diacritics:(.*?)- Ō",
r"possible interpretation with diacritics:(.*?)This",
r"write it as:(.*?)Here",
r"accent marks:(.*?)Here",
r"the appropriate diacritics and accent marks:(.*?)Here's",
r"Here is the text with diacritics:(.*?)Here's",
r"Here is the text with diacritics:(.*?). In",
r"diacritics and accent marks:(.*?)Here",
r"with appropriate diacritics:(.*?)-",
r"diacritics and accent marks added:(.*?)In",
r"diacritics and accent marks added:(.*?).",
r"write it as follows:(.*?)Here",
r"should be written as:(.*?)Here",
r"text with diacritics:(.*?).",
r"text with diacritics:(.*?)Here",
r"diacritics:(.*?).",
r"with diacritics and accent marks:(.*?)-",
r"with diacritics and accent marks added:(.*?).",
r"Here is a version with diacritics:\s*\n+(.*?)\n",
r'Here is the text with diacritics and accent marks:\s*\n*(.*?)"',
r"إلى النص كالتالي:\s*\n+(.*?)\n",
r"Here is the text with diacritics and accent marks added:\s*\n+(.*?)\n",
r"當然可以，以下是帶有標點符號和重音符號的文本:\s*\n+(.*?)\n",
r"Here is the text with diacritics and accent marks:\s*\n+(.*?)\n",
r"Here's the translation with diacritics and accent marks:\s*\n+(.*?)\n",
r"with diacritics and accent marks:\s*\n+(.*?)\n",
r" مع الإشارة التائيّة والضاد والفتحة هي:\s*\n+(.*?)\n",
r"Here's the result:\s*\n+(.*?)\n",
r"diacritics and accent marks in Arabic is:\s*\n+(.*?)\n",
r"with diacritics and accents:\s*\n+(.*?)\n",
r"with diacritics and accent marks added is:\s*\n*(.*)",
r"with diacritics and accent marks is:\s*\n*(.*)",
r"with added diacritics and accent marks is:\s*\n+(.*?)\n",
r"Here is the text with added diacritics and accent marks:\s*\n+(.*)",
r"Here is one possible way to do it:\s*\n*(.*)",
r"you would write it as:\s*\n*(.*)",
r"in Arabic script is:\s*\n*(.*)",
r"diacritics and accent marks added:\s*\n+(.*?)\n",
r"Here it is with diacritics added for clarity:\s*\n+(.*?)\n",
r"Here is the text with the full diacritics:\s*\n+(.*?)\n",
r"Here is the text with diacritics:\s*\n+(.*?)\n",
r"Here is a possible version with diacritics:\s*\n+(.*?)\n",
r"Here it is again for clarity:\s*\n+(.*?)\n",
r"Here is one possible version with diacritics:\s*\n+(.*?)\n",
r"Here's the text with diacritics:\s*\n+(.*?)\n",
r"The text with diacritics and accent marks in Arabic is:\s*\n+(.*?)\n",
r"Here's a possible version with diacritics:\s*\n+(.*?)\n",
r"it would be written as:\s*\n+(.*?)\n",
r"it would look like this:\s*\n+(.*?)\n",
r"Here's one possible way to do it:\s*\n+(.*?)\n",
r"The text with diacritics and accent marks is:\s*\n*(.*)",
r"Here is the text with diacritics added for clarity:\s*\n+(.*?)\n",
r"The full phrase with diacritics would be:\s*\n+(.*?)\n",
r"Here is your text with optional diacritics added for clarity:\s*\n+(.*?)\n",
r"Here's the text with diacritics and accent marks(?: added)?:\s*\n*(.*)",
r'With diacritics and accent marks added, it becomes:\s*(.*?)\n\n',
r'\*\*With Diacritics and Accents \(for emphasis\):\*\*\s*(.*?)\n\n',
r'Here’s the sentence with added diacritical marks for clarity:\s*(.*?)\n\n',
r'with added diacritics and accent marks is:\s*(.*?)\n\n',
r'with added diacritics and accent marks:\s*(.*?)\n\n',
r'with diacritics as follows:\s*(.*?)\n\n',
r'With diacritics and accents:\s*(.*?)\n\n',
r'With diacritics and accent marks:\s*(.*?)\n\n',
r"here's how you might write it:\s*(.*?)\n\n",
r'correct diacritics added:\s*(.*?)\n\n',
r'proper tones and accents as follows:\s*(.*?)\n\n',
r'out in its Yoruba form:\s*(.*?)\n\n',
r'The corrected sentence might be:\s*(.*?)\n\n',
r'The sentence with diacritics is:\s*(.*?)\n\n',
r'look something like this:\s*(.*?)\n\n',
r'the sentence with diacritics added:\s*(.*?)\n\n',
r'sentence with the correct diacritics:\s*(.*?)\n\n',
r'With diacritics added for proper tones:\s*(.*?)\n\n',
r'correct diacritics and accents would be:\s*(.*?)\n\n',
r'depending on the tonal rules of Yoruba:\s*(.*?)\n\n',
r'sentence with the appropriate diacritics:\s*(.*?)\n\n',
r'is properly marked as ""bicycle"":\s*(.*?)\n\n',
r'spelled out in its Yoruba form:\s*(.*?)\n\n',
r'including the correct spelling of ""olori"":\s*(.*?)\n\n',
r'appropriate diacritics added:\s*(.*?)\n\n'
r'with added diacritics and accent marks:\s*(.*?)\n\n',
r'With diacritics added for emphasis:\s*(.*?)\n\n',
r'The corrected sentence might be:\s*(.*?)\n\n',
r'with diacritics as follows:\s*(.*?)\n\n',
r'he correct spelling of ""olori"":\s*(.*?)\n\n',
r' with added diacritics and accent marks would be:\s*(.*?)\n\n',
r'Sure, here is the text with diacritics and accent marks:\s*(.*?)\n',
r'The text should be modified to include diacritics and accent marks as follows:\s*(.*?)\n',
r"Here's the text with diacritics and accent marks added:\s*(.*?)\n\n",
r"Here's the text with diacritics and accent marks added: \s*(.*?)\n",
r"Here's the text with diacritics and accent marks:\s*(.*?)\n",
r"Here's the text with diacritics and accent marks(?: added)?:\s*\n*(.*)"
r"Here is the text with diacritics:\s*\n+(.*?)\n",
r"Here is the text with optional diacritics added for clarity:\s*\n+(.*?)\n",
r"you would write it as follows:\s*\n+(.*?)\n",]

In [ ]:
def extract_answer(text):    
    for m in matches_qwen:
        if re.search(m, text, re.DOTALL):
            return re.search(m, text, re.DOTALL).group(1).replace("*", " ").replace("`", "").replace('"', " ").strip()
    return text.replace("*", "").replace("`", " ").replace('"', " ").strip()

### Sample usage

In [ ]:
df_qwen = pd.read_csv("/l/users/hawau.toyin/NLP804/_finetune_results/_finetune_results/Qwen2/test.csv", sep="\t")
df_qwen['response'] = df_qwen['response'].apply(extract_answer)
df_qwen.to_csv("/l/users/hawau.toyin/NLP804/_finetune_results/_finetune_results/Qwen2/test_.csv", sep="\t", index=False)
df_qwen.head()

,sentence,undiacritized,normalized,root,root_meaning,response
0,Ọ̀mọ̀ pọ̀ nínú igbó láti gé,Òmò pò ninu igbo lati ge,òmò pò ninu igbo lati ge,Ọ̀mọ̀,Tree of the banyan family,Òmò pò nínú ígbò láti gé-ẹ̀rú.Here's the text ...
1,Gbogbo ẹni ti o ṣe owò ni yíò jèrè rẹ̀,Gbogbo eni ti o se owo ni yio jere rè,gbogbo eni ti o se owo ni yio jere rè,Owò,business,
2,Ọmọ to bá ṣe iṣẹ́ dédé dandan ni kí ó sinmi,Omo to ba se isé dede dandan ni ki o sinmi,omo to ba se isé dede dandan ni ki o sinmi,Iṣẹ́,work,
3,Mo fẹ́ se ìrìn-àjò ní Ọ̀sán òní,Mo fé se irin-ajo ni Òsan oni,mo fé se irin ajo ni òsan oni,Ọ̀san,Afternoon,
4,Òkun lolórí omi.,Okun lolori omi.,okun lolori omi,Òkun,Ocean.,Ọkụnyị lotte ọmụma.Explanation:- Okun become...


In [ ]:
df = pd.read_csv("//l/users/hawau.toyin/NLP804/_finetune_results/_finetune_results/Qwen2/test_.csv", sep="\t", on_bad_lines='skip')
model = "response"
print(f"Evaluating {model}")
df[model] = df[model].apply(lambda x: remove_punctuation(str(x)))
wer_score = wer.compute(references=df['normalized'].tolist(), predictions=df[model].tolist())
cer_score = cer.compute(references=df['normalized'].tolist(), predictions=df[model].tolist())
print(f"WER: {100 * wer_score:.2f}, CER: {100 * cer_score:.2f}")

Evaluating response
WER: 84.18, CER: 35.34
